<a href="https://colab.research.google.com/github/Takumi173/JPMA2023TF1-1/blob/main/JPMA2023_case3_(2)_LLM%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E5%8A%B9%E8%83%BD%E3%81%BE%E3%81%9F%E3%81%AF%E5%8A%B9%E6%9E%9C%E3%81%AE%E7%A2%BA%E8%AA%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続とパスの指定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# データが保存されたディレクトリの指定
datadir = '/content/drive/MyDrive/JPMA2023_case3/'


Mounted at /content/drive


## ライブラリとパッケージのインストール

In [2]:
!pip install transformers bitsandbytes accelerate

import pickle
import torch
import gc
import pandas as pd
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm.notebook import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 20.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

## 添付文書データのロード

In [3]:
with open(datadir+'Label_JP.pkl', 'rb') as f:
  Label_JP = pickle.load(f)

## LLMモデルのロード

In [4]:
# HuggingFaceのログイン
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# 使用するモデルの指定
model_id = 'meta-llama/Llama-2-13b-chat-hf'

In [6]:
# Tokenizerのロード
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 4bitでモデルをロード
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=bnb_config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# 効能または効果の確認

## 関数定義

In [7]:
# 成分名が与えられたときに効能効果を取得する関数
def getIndications_Ja(searchName, wurl = False):

  # 成分名が一致するレコードを抽出
  tgt = Label_JP.query(f'成分名 == "{searchName}"').values.tolist()


  # 一致するレコードがない場合
  if len(tgt) == 0:
    return "該当する薬がありません"

  # 一致するレコードがある場合

  context = ""
  Type = ""

  for i, t in enumerate(tgt):

    # 区分を追加（区分が変わるごとに追加する）
    if Type != t[1]:
      Indc = ""
      Type = t[1]
      context += f"【{Type}の場合】\n"

    # 効能効果を追加（効能効果が変わるごとに追加する）
    if Indc != t[2]:
      Indc = t[2]
      context += Indc + "\n"

      # PMDA添付文書へのURLを生成する（wurl == Trueが与えられた場合）
      if wurl == True:
        lnk = t[4].split(";\n")
        for pa in list(set(lnk)):
          context += "(https://www.info.pmda.go.jp/go/pack/" + pa + "/?view=frame&style=XML&lang=ja)"+ "\n"

    context +=  "\n"
  return context[:-1].replace(";","")  # 最後の改行と余分なセミコロンを削除して返す


In [8]:
# GPU memory flusher
def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  return

# Generator
def Generator (medication, indication):
  context = getIndications_Ja(medication)

  prompt = f"""<s>[INST] <<SYS>>
You are an excellent physician.  Please answer the questions as accurately as possible based on the medical knowledge you have.  Take a deep breath, read the text carefully, and think step-by-step!
<</SYS>>

### Context
The followings are the indications of a certain drug:
{context}

### Quesitons
Step 1: List all disease(s) or symptom(s) the drug effective.
Step 2: Translate '{indication}' in English.
Step 3: Based on the above, is this drug effective for '{indication}' itself?  Start your answer with Yes or No.

### Answer
[/INST] """

  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    # メモリ上限を超えないためのトークン数キャップ
    if len(token_ids[0]) > 1500:
      output = "Too many text to read"
      return prompt, output

    output_ids = model.generate(
      token_ids.to(model.device),
      max_new_tokens=1024,    # 生成するメッセージの最大トークン数
      do_sample=False,        # ランダム生成をしない
    )

  # 生成メッセージの取得
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

  return prompt, output   # プロンプトと生成メッセージを返す

## テスト生成

In [9]:
prompt, output = Generator("アジルサルタン・アムロジピンベシル酸塩", "本態性高血圧")

print("*** OUTPUT ***")
print(output)
print("\n*** PROMPT ***")
print(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


*** OUTPUT ***
Sure, I'd be happy to help!

Step 1: List all disease(s) or symptom(s) the drug effective.

Based on the indications provided, the drug is effective for:

* High blood pressure (hypertension)

Step 2: Translate '本態性高血圧' in English.

'本態性高血圧' can be translated to "essential hypertension" in English.

Step 3: Based on the above, is this drug effective for '本態性高血圧' itself?

Yes, the drug is effective for essential hypertension.

*** PROMPT ***
<s>[INST] <<SYS>>
You are an excellent physician.  Please answer the questions as accurately as possible based on the medical knowledge you have.  Take a deep breath, read the text carefully, and think step-by-step!
<</SYS>>

### Context
The followings are the indications of a certain drug:
【内用薬の場合】
高血圧症


### Quesitons
Step 1: List all disease(s) or symptom(s) the drug effective.
Step 2: Translate '本態性高血圧' in English.
Step 3: Based on the above, is this drug effective for '本態性高血圧' itself?  Start your answer with Yes or No.

### Answe

## 効能または効果が取得可能な成分に対する特定疾患の有効性確認

In [10]:
# すべての成分に対して調査をかけるには時間がかかりすぎるため分割実行が必要

SortedList = sorted(list(set(Label_JP["成分名"].values.tolist())))

SampleLists = SortedList[:10]
#SampleLists = SortedList[:300]
#SampleLists = SortedList[300:600]
#SampleLists = SortedList[600:900]
#SampleLists = SortedList[900:1200]
#SampleLists = SortedList[1200:1500]
#SampleLists = SortedList[1500:]

# 調査対象の疾患等
TargetIndication = "糖尿病"


In [11]:
# Warningを表示しない
warnings.simplefilter('ignore')

# 確認する成分名と対象疾患、生成したプロンプト及び得られた結果、使用した添付文章上の効能または効果をDataFrameにまとめる
preds = []
for ActiveIngredient in tqdm(SampleLists):
  pred = []

  prompt, output = Generator(ActiveIngredient, TargetIndication)

  pred.append(ActiveIngredient)
  pred.append(TargetIndication)
  pred.append(prompt)
  pred.append(output)
  pred.append(getIndications_Ja(ActiveIngredient, wurl=True))
  preds.append(pred)


preds_df = pd.DataFrame(preds, columns=["成分名", "Target Indication", "Prompt", "Response", "添付文書上の効能または効果"])
preds_df.to_csv("Results_"+model_id.replace("/","_")+".csv")
preds_df

  0%|          | 0/10 [00:00<?, ?it/s]

,成分名,Target Indication,Prompt,Response,添付文書上の効能または効果
0,β－ガラクトシダーゼ（アスペルギルス）,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\n乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不...
1,β－ガラクトシダーゼ（ペニシリウム）,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\n乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不...
2,きゅう帰膠艾湯エキス,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\n冷え症で、出血過多により、貧血するもの。痔出血、外傷後の内出血、産後出血...
3,きゅう帰調血飲エキス,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\n産後の神経症、体力低下、月経不順\n(https://www.info....
4,よく苡仁湯エキス,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\n関節痛、筋肉痛\n(https://www.info.pmda.go.j...
5,アカラブルチニブ,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\n慢性リンパ性白血病（小リンパ球性リンパ腫を含む）\n(https://w...
6,アカンプロサートカルシウム,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【内用薬の場合】\nアルコール依存症患者における断酒維持の補助\n(https://www....
7,アガルシダーゼアルファ（遺伝子組換え）,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【注射薬の場合】\nファブリー病\n(https://www.info.pmda.go.jp...
8,アガルシダーゼベータ（遺伝子組換え）,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【注射薬の場合】\nファブリー病\n(https://www.info.pmda.go.jp...
9,アガルシダーゼベータ（遺伝子組換え）［アガルシダーゼベータ後続１］,糖尿病,<s>[INST] <<SYS>>\nYou are an excellent physic...,"Sure, I'd be happy to help!\n\nStep 1: List al...",【注射薬の場合】\nファブリー病\n(https://www.info.pmda.go.jp...
